# HYDRO30 Surface Water Extent Map: Process RTC stack

This notebook processes a stack of HYDRO30 products from a stack of HyP3 or OPERA dual-pol RTCs.

In [1]:
from datetime import datetime
from pathlib import Path

import geopandas as gpd
from hydrosar.water_map import make_water_map
from ipyfilechooser import FileChooser
import sys
from tqdm.auto import tqdm

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

---
## 1. Select a directory holding an RTC stack prepared with the [Prepare a SAR RTC Data Stack for HydroSAR notebook](1a_Prepare_HydroSAR_RTC_Stack.ipynb)

The directory should contain the following subdirectories:
- `VH`
- `VV`

In [2]:
fc = FileChooser(Path.home())
display(fc)

FileChooser(path='/home/jovyan', filename='', title='', show_hidden=False, select_desc='Select', change_desc='…

In [3]:
data_dir = Path(fc.selected_path)
vh_dir = data_dir / 'VH'
vv_dir = data_dir / 'VV'

vh_paths = sorted(list(vh_dir.glob('*VH*.tif')))
vv_paths = sorted(list(vv_dir.glob('*VV*.tif')))

water_mask_dir = data_dir / 'Water_Mask'
water_mask_dir.mkdir(exist_ok=True)

In [4]:
gdf = gpd.GeoDataFrame(
    {
    'file': vh_paths + vv_paths,
    'data_type': ['VV_RTC' if 'VV' in str(pth) else 'VH_RTC' for pth in vh_paths + vv_paths],
    'SAR_acquisition_date': util.get_dates(vh_paths + vv_paths),
    'EPSG': [util.get_epsg(p) for p in vh_paths + vv_paths],
    'geometry': [util.get_geotiff_bbox(p) for p in vh_paths + vv_paths],
    }
                  )
display(gdf)
if gdf['EPSG'].nunique() > 1:
    raise Exception(f'Error: Data is in {gdf["EPSG"].nunique()} projections.')

,file,data_type,SAR_acquisition_date,EPSG,geometry
0,/home/jovyan/bang2/VH/S1A_IW_20180709T234718_D...,VH_RTC,2018-07-09,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
1,/home/jovyan/bang2/VH/S1A_IW_20180721T234719_D...,VH_RTC,2018-07-21,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
2,/home/jovyan/bang2/VH/S1A_IW_20180802T234719_D...,VH_RTC,2018-08-02,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
3,/home/jovyan/bang2/VH/S1A_IW_20180814T234720_D...,VH_RTC,2018-08-14,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
4,/home/jovyan/bang2/VH/S1A_IW_20180826T234721_D...,VH_RTC,2018-08-26,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
5,/home/jovyan/bang2/VH/S1A_IW_20180907T234721_D...,VH_RTC,2018-09-07,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
6,/home/jovyan/bang2/VH/S1A_IW_20180919T234722_D...,VH_RTC,2018-09-19,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
7,/home/jovyan/bang2/VH/S1A_IW_20181001T234722_D...,VH_RTC,2018-10-01,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
8,/home/jovyan/bang2/VV/S1A_IW_20180709T234718_D...,VV_RTC,2018-07-09,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."
9,/home/jovyan/bang2/VV/S1A_IW_20180721T234719_D...,VV_RTC,2018-07-21,32646,"POLYGON ((385650.000 2691600.000, 498090.000 2..."


---
## 2. Generate the water extent maps

In [5]:
%%time
for d in tqdm(set(gdf['SAR_acquisition_date'])):
    vh_raster = gdf.loc[(gdf['SAR_acquisition_date'] == d) & (gdf['data_type'] == 'VH_RTC')]['file'].iloc[0]
    vv_raster = gdf.loc[(gdf['SAR_acquisition_date'] == d) & (gdf['data_type'] == 'VV_RTC')]['file'].iloc[0]

    water_extent_raster = water_mask_dir / f'water_extent_{datetime.strftime(d, "%Y%m%d")}.tif'

    make_water_map(
        water_extent_raster, 
        vv_raster, 
        vh_raster, 
        tile_shape=(100, 100),
        max_vv_threshold=-15.5, 
        max_vh_threshold=-23., 
        hand_threshold=15., 
        hand_fraction=0.8
    )

  0%|          | 0/8 [00:00<?, ?it/s]

Threshold too high! Using maximum threshold -15.5 db


CPU times: user 1min 26s, sys: 6.46 s, total: 1min 32s
Wall time: 2min 34s


*HYDRO30_Stack_Processing.ipynb - Version 1.0.0 - May 2024*